# Custom data

In [11]:
import re
from sklearn.model_selection import train_test_split

# Script parses RAW_INPUT_PATH, which is a dump from dynalist, and outputs
# two files: OUTPUT_TRAIN_PATH, OUTPUT_TEST_PATH, which are preprocessed sentences, split into train and test
RAW_INPUT_PATH = 'data/research_dump.txt'
TRAIN_PATH = 'data/train.txt'
TEST_PATH = 'data/test.txt'
MODEL_NAME = 'gpt2'

"""
@param sentence: string.
@return bool. Whether or not the sentence should be included as part of training.
"""
def is_valid_sentence(sentence):
    return len(sentence) > 10

"""
@param sentence: string.
@return string. The preprocessed sentence.
"""
def preprocess_sentence(sentence):
    return re.sub(r"\*\*", "", sentence).strip()

sentences = []
sentence_cnt = {}
line_id = 0
with open(RAW_INPUT_PATH, encoding='utf-8') as f:
    for line in f:
        line_id += 1
        line_sentences = line.split("\n")
        num_line_sentence = len(line_sentences)
        sentence_cnt[num_line_sentence] = sentence_cnt.get(num_line_sentence, 0) + 1
        for sentence in line_sentences:
            # print(f"{line_id}: {sentence}")
            sentence = preprocess_sentence(sentence)
            if not is_valid_sentence(sentence):
                continue
            sentences.append(sentence)
print(sentence_cnt)
train_sentences, test_sentences = train_test_split(sentences, test_size=0.10)

with open(TRAIN_PATH, "w", encoding='utf-8') as outfile:
    outfile.write("\n".join(train_sentences))
    
with open(TEST_PATH, "w", encoding='utf-8') as outfile:
    outfile.write("\n".join(test_sentences))

{2: 16443, 1: 1}


In [12]:
# 15K sentences
print(len(sentences))

15088


# Fine tuning

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [6]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=4)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=4)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset(TEST_PATH, TEST_PATH, tokenizer)

In [7]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

MODEL_OUTPUT_DIR = "model/finetuned-gpt2"
model = AutoModelWithLMHead.from_pretrained(MODEL_NAME)

training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_DIR, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    per_device_eval_batch_size=1,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)

C:\Users\sjona\anaconda3\lib\site-packages\transformers\modeling_auto.py:821: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
C:\Users\sjona\anaconda3\lib\site-packages\transformers\trainer.py:259: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  warnings.warn(


# Train the model

In [8]:
trainer.train()

Step,Training Loss
500,6.203875
1000,6.108259
1500,6.001910
2000,5.939924
2500,6.023035
3000,5.874520
3500,5.874719
4000,5.739141
4500,5.800875
5000,5.816199


TrainOutput(global_step=23325, training_loss=4.615244172025723)

In [9]:
trainer.save_model()

# Text generation demo

In [44]:
from transformers import pipeline


MODEL_OUTPUT_DIR = "model/finetuned-gpt2"

def generate_sentences(prompt, temperature, num_sample):
    """
    @param prompt: string. The sentence to autocomplete.
    @param temperature: float. Higher makes the model more indecisive. 
    @param num_sample: int. How many sentences to generate.
    @return string[]. List of num_sample sentences.
    """
    
    # See https://huggingface.co/transformers/main_classes/configuration.html#transformers.PretrainedConfig
    # For some reason, num_return_sequences is ignored.
    config = {
        'max_length': 1500,
        'do_sample': True,
        'top_k': 50,
        'temperature': temperature
    }
    generator = pipeline('text-generation',model=MODEL_OUTPUT_DIR, tokenizer='gpt2',config=config)
    
    sentences = []
    for i in range(num_sample):
        sentence = generator(prompt)[0]['generated_text']
        sentences.append(sentence)
    return sentences

In [45]:
def write_sentence(filewriter, sentence):
    filewriter.write(sentence)
    print(sentence)

In [46]:
prompts = [
"Neurons are",
"The brain is",
"The number of neurons in the brain is",
"The universe is",
"Math is",
"Science is",
"The world is",
"What is the next big thing?",
"Information should be",
"I like"    
]

NUM_SAMPLE = 5
GENERATED_SENTENCES_PATH = "data/generated_sentences.txt"
with open(GENERATED_SENTENCES_PATH, "w", encoding='utf-8') as outfile:
    for prompt in prompts:
        for temperature in [0.5, 0.7, 0.9]:
            write_sentence(outfile, "-------------------------------\n")
            write_sentence(outfile, f"Temperature: {temperature}. Prompt = {prompt}\n")
            sentences = generate_sentences(prompt, temperature, NUM_SAMPLE)
            for sentence in sentences:
                write_sentence(outfile, sentence + "\n")
    

-------------------------------

Temperature: 0.5. Prompt = Neurons are



Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Neurons are assumed to have same function
Neurons show clear neural function but different functions
Sensory neurons vs non-neural: no V(t) neurons, only V(t) neurons and V(t) are

Neurons are dominant Neurons, Neurons are non- dominant Neurons, Neurons dominant, Neural, Neurons dominant, Neural, Neurons dominant Neurons dominant Neurons dominant, Ne

Neurons are interneuron-species. "Neural activity is directly interneuron-species in decision-making in Ca2+-dependent"
Neurons are decision making with Ca2+- insensitive, and non-ne

Neurons are organized into 1 sub-sect-brain, sub-sect-brain-behavioral, interneuron-level dynamics
Neural, interneuron-level, interneuron. Temporal-level interneuron-

Neurons are not interneuron-based
Data extraction. Neurons are generated by using F(L, Neurons and their interneuron model. Each neuron has its own postsynaptic input is its own Neurons

-------------------------------

Temperature: 0.7. Prompt = Neurons are



Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Neurons are more interconnected, there is the interneuron model with the network
Neurons are interneuron model: Neurons connectome connectome with interneuron.
Neural models, not necessarily necessary for causality

Neurons are strongly expressed in Na+- insensitive Neurons
Neural response, Neurons strongly expressed in Vth response
Neurons expressed in Ca+ insensitive. Neural response to Vth response, Vth response for

Neurons are connected to each motor neuron, as opposed to non-neurons
Human knowledge discovery process very much in a monotonous pathway, eigen-controllability-based. Higher levels more structured brain
More complex patterns

Neurons are involved in the multi-modal system
Decision making mechanisms at the system-level. Neuron-level. Closed loop, multi-modal, multi-modal decision making: Neural decision-making making during

Neurons are assumed to be causality relationships
Multi-modal relationships among different ontologies
Neurons and monotonous Neural Vth (neural, s

...

I like Q: What will Q: Would you like to have Q: Would you like Q: Would you like the final state to have its own sentence structure
I like your formulation. It's a summary of the definition of semantic terms. Q

I like how they pick them. I like the picker.
My preference. I don't see any 2 options because of the focus on 1 output. If interested, I can experiment and compare them too. Papers 1 and 2 should start on

I like the framework for presenting my papers. I think it's simple enough that its usefulness.
Open question --- I like how many examples that open question. Can't find references -- I will probably change over time. https://docs.google.

I like them so much more options, but I find that they don't seem more intuitive but still. I willymu terms too. I like reading intro but unfamiliarity to reading
Key findings: I like having intuitive interneuron vs inter

I like the idea of hierarchy, but don't like hierarchy too fixed. I like the framework - hierarchy: hierarchical syste

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


I like the idea of it.
Need more examples. Need more examples and comments. Craving it. Go through all the examples. https://docs.google.com/academic.com/reference/d/1149f55.

I like Kato's example - can use Kato's example first - can't just assume that my objective is objective is to justify my decision making standards.
I would like how you guys think it - Kato's proposal - you should just

I like this idea of having this analogy because it's easy to visualize the functional connections between neurons, and not just behavioral
Also worth noting that this: I like the conceptualizations. Just to get something out there that's informative when it goes towards

I like reading patterns, but I get what I want.
One that I like the framework. It's nice that everyone has a framework and I can think of thinking about using this. I get the idea that if I want to start thinking about

I like reading https://docs.google.com/scipy.com/acom/author/shliht.com/adam.com/~demarcation/scipy.com/referenc

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


I like thinking about moving forward. For now. I want to experiment. I think that much easier over time. For now, just learn weights and measures. I_ext - useful to have background knowledge base. I want to dig my own conclusions

I like how they break down causality relationships pretty crude to model. I like the framework of categorization.
Decision making
Just to show how many causality relationships: 2 (I think it's just categorize organisms -- organism vs cy

I like / dislike modulatory. I don't need a reference. Too interconnected wikipedia
If more than 1: insufficient, I will dismiss potential conflicts. Maybe I need a proposal by Kato: "completes existing conflicts".
Suggest

I like XML - good fit. I like ML for ML for structured presentations.
This is a good fit because it allows for single-use presentations. I was thinking - I like structured slides. I like categorize slides. I like categor

I like Eli's decision making approach, but I don't think it's wise for Eli's decision